In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('nlp').getOrCreate()
spark

In [ ]:
categories = spark.read.parquet("../../data/sklep/categories")
categories.printSchema()

In [ ]:
customers = spark.read.parquet("../../data/sklep/customers")
customers.printSchema()

In [ ]:
customers.show(5)

In [ ]:
departments = spark.read.parquet("../../data/sklep/departments")
departments.printSchema()
departments.show(5)

In [ ]:
order_items = spark.read.parquet("../../data/sklep/order_items")
order_items.printSchema()
order_items.toPandas().head(5)

In [ ]:
orders = spark.read.parquet("../../data/sklep/orders")
orders.printSchema()
orders.head(5)

In [ ]:
products = spark.read.parquet("../../data/sklep/products")
products.printSchema()
products.head(5)

In [ ]:
join_types = ["inner", "cross", "outer", "full", "left_outer", "right_outer", "left_semi", "left_anti"]

In [ ]:
categories.join(departments, categories.category_department_id == departments.department_id).show(5)

In [ ]:
for join_type in join_types:
    cats_on_deps = categories.join(other=departments, on=categories.category_department_id == departments.department_id, how=join_type)
    count = cats_on_deps.count()
    print(join_type, count)

In [ ]:
order_items.join(products, order_items.order_item_product_id == products.product_id).show(5)

In [ ]:
spark

In [ ]:
customers.createOrReplaceTempView("customers")

In [ ]:
df = spark.sql("SELECT * FROM customers")

In [ ]:
df.show(5)

In [ ]:
df.toPandas().head(5)

In [ ]:
data_frames = [categories, departments, order_items, orders, products]
data_frames_names = ["categories", "departments", "order_items", "orders", "products"]
for df, df_name in zip(data_frames, data_frames_names):
    df.createOrReplaceTempView(df_name)

In [ ]:
spark.sql("select * from categories").show(5)

In [ ]:
spark.sql("select category_id, category_name from categories").show(5)

In [ ]:
spark.sql("select distinct category_id, category_name from categories where category_id = 1").show(5)

In [ ]:
with open("../../src/sql/select_category.sql", "r") as f:
    category_query_template = f.read()


In [ ]:
category_query_template

In [ ]:
category_query_template.format(category_id=1)

In [ ]:
spark.sql(category_query_template.format(category_id=1)).show(5)

In [ ]:
def get_query_template(query_name: str) -> str:
    with open(f"../../src/sql/{query_name}.sql", "r") as f:
        return f.read()

def get_query(query_name: str, **kwargs) -> str:
    return get_query_template(query_name).format(**kwargs)

In [ ]:
spark.sql(get_query("select_category", category_id=1)).show(5)

In [ ]:
orders.show(5)

In [ ]:
orders.select("order_status").distinct().show()

In [ ]:
spark.sql("select distinct order_status from orders").show()

In [ ]:
df = spark.sql("select distinct order_status from orders")
df

In [ ]:
df.collect()

In [ ]:
df.collect()[0]

In [ ]:
df.collect()[0].order_status

In [ ]:
df.collect()[0]["order_status"]

In [ ]:
df.collect()[0][0]

In [ ]:
df.collect()[0].asDict()

In [ ]:
df = spark.sql("select distinct order_status from orders order by order_status")
df.show()

In [ ]:
orders.select("order_status").distinct().orderBy("order_status").show()

In [ ]:
query = """
select c.category_name, count(order_item_quantity) as count
from order_items oi
inner join products p on oi.order_item_product_id = p.product_id
inner join categories c on c.category_id = p.product_category_id
group by c.category_name
order by count desc
limit 10;
"""

In [ ]:
spark.sql(query).show()

In [ ]:
query = get_query("top_categories")
query

In [ ]:
spark.sql(query).show()

In [ ]:
query = """
select
    oi.order_item_product_id,
    sum(cast(oi.order_item_subtotal as float)) as revenue
from order_items oi
inner join orders o on oi.order_item_order_id = o.order_id
where o.order_status <> 'CANCELED' and o.order_status <> 'SUSPECTED_FRAUD'
group by order_item_product_id;
"""

In [ ]:
spark.sql(query).show()

In [ ]:
spark

In [ ]:
categories.write.mode("overwrite").parquet("../../data/categories.parquet")

In [ ]:
customers.write.mode("overwrite").csv("../../data/customers.csv")